#### Cleaning and OHE for multilabel categorical variables  <br>
Sebastian Deimen - MS Info Capstone - Univer

In [1]:
import pandas as pd, json, pickle as pkl, numpy as np, pickle
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
with open("data/all_opportunities_df.pkl", 'rb') as infile:
    df = pkl.load(infile)

### Varibales PreProcessing/String cleaning/replacing

In [3]:
# None values in AGES, replacing it with "all ages"
df.ages[df.ages.isna()]
df.ages.iloc[341] = ["All Ages"]
df.ages.iloc[341]

['All Ages']

In [7]:
# Prefix Ages and manipulate and unify the ages group and fix an age type (9 to 6)
for i,el in enumerate(df.ages,1):
    df.loc[i,'ages'] = ["Ages_Post Secondary" if (age == "Post Secondary School") 
                        else "Ages_Elementary (4-6)" if age == 'Grades 4-9' else "Ages_" + age for age in el if age != '']       

In [5]:
# Unify REACH
df.reach = df.reach.apply(lambda x: 'Missouri' if x == ['Missouri', 'Illinois'] else x)
df.reach = df.reach.apply(lambda x: np.nan if x == [] else x)
df.reach = df.reach.apply(lambda x: str(x).strip("[]''"))
df.reach = df.reach.apply(lambda x: np.nan if x == "None" else x)

In [6]:
# Replacing/abbreviating strings for ATTENTION
df.attention = df.attention.apply(lambda x: ['None'] if x == [] or x == None else x)
for i,row in enumerate(df.attention,1):
    df.loc[i,'attention'] = ["Att_" + "GTS" if (el == 'Gifted and talented students' 
                                                or el == 'Gifted and Talented') else "Att_" + "SwDis" if (el == 'Students with disabilities' 
                                                or el == 'Students with disabilites') else "Att_" + "SRDoS" 
                                                if el == 'Students at risk from dropping out of school' else "Att_" + el for el in row]


In [7]:
# LANGUAGE unifying 
df.language = df.language.apply(lambda x: ['None'] if x == [] or x == None else x)
for i, row in enumerate(df.language,1):
    df.loc[i,"language"] = ["L_" + el for el in row]

In [8]:
# Taking care of ATTENTION - replacing the one None Value with Data found in the data source
df.areaOfInterest.loc[342] = ['Coding/Programming', 'Biology','General Science', 
                              'Invention Education', 'Making/DIY', 'Computer Science', 'Space Science/Aerospace/Rocketry', 
                              'Technology', 'Entrepreneurship', 'Engineering', 'Earth and Environmental Science', 'Robotics', 'Math']

In [9]:
# Replacing/abbreviating strings in ATTENTION
for i,row in enumerate(df.areaOfInterest,1):
    df.loc[i,'areaOfInterest'] = ["AoI_EES" if (el == 'Earth and Environmental Science' or el == 'earth and environmental science' or el == 'earth and Environmental Science') 
                                  else "AoI_CS" if (el == 'Computer Science' or el =='computer science') else "AoI_" + 'Coding/Programming' if el == 'coding/programming' 
                                  else "AoI_GS" if el == 'General Science' else "AoI_AaD" if el == 'Architecture and Design' 
                                  else "AoI_SSAR" if el == 'Space Science/Aerospace/Rocketry'                                  
                                  else "AoI_" + el for el in row]


In [10]:
# TYPEOFOPPORTUNITY stripping the list brackets
df.typeOfOpportunity = df.typeOfOpportunity.apply(lambda x: str(x).strip("[]'"))

### One Hot Encodings

In [11]:
# run the OHE on AGES, ATTENTION, LANGUAGE, AREAOFINTEREST
mlb = MultiLabelBinarizer()
variables = [df.ages,df.attention, df.language, df.areaOfInterest]
for el in variables:
     df = df.join(pd.DataFrame(mlb.fit_transform(el),
                          columns=mlb.classes_,
                          index=df.index))

### not running OHE on reach for now - everything is only one level

In [40]:
# run the OHE on reach
# mlb = MultiLabelBinarizer()
# df = df.join(pd.DataFrame(mlb.fit_transform(df.reach),
                         # columns=mlb.classes_,
                          # index=df.index))

### Varibales Checking

In [ ]:
for n,el in enumerate(df.reach,1):
    print(n, el)

In [ ]:
df.reach[df.reach.isna()]

In [ ]:
df.attention[df.attention.isna()]


In [ ]:
for n, el in enumerate(df.attention,1):
    print(n, el)

In [ ]:
for n, el in enumerate(df.language,1):
    print(n, el)

In [ ]:
for n, el in enumerate(df.areaOfInterest,1):
    print(n, el)

In [ ]:
for n, el in enumerate(df.areaOfInterest,1):
    print(n, el)

### Store the data

In [12]:
with open("cleaned_data.pkl", "wb") as outfile:
    pickle.dump(df, outfile)


In [13]:
df.to_csv("data/all_opportunities_df_cleaned.csv")